# Group Exam INFO284, First Draft
This is a first draft of the group exam, made by Ingeborg Glad, Hans Eirik Opsahl, Sindre Froestad and Endre Bjørgo

## Task 1: Sentiment Analysis
In this first task the goal is to create 4 different models which are able to classify the sentiment in hotel reviews. The 4 models require the following long list of imports:

In [2]:
import pandas as pd
import numpy as np
import ast
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, \
     median_absolute_error, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import mixed_precision

<class 'ModuleNotFoundError'>: No module named 'seaborn'

In addition, the data must be prepared correctly.

In [3]:
data = pd.read_csv('Hotel_Reviews.csv')

# Dropping all columns considered irrelevant
data.drop('Hotel_Address',axis=1,inplace=True)
data.drop('Review_Date',axis=1,inplace=True)
data.drop('Additional_Number_of_Scoring',axis=1,inplace=True)
data.drop('lat',axis=1,inplace=True)
data.drop('lng',axis=1,inplace=True)
data.drop('Total_Number_of_Reviews',axis=1,inplace=True)

# Splitting the 'Tags' column into multiple columns
data['Tags'] = data['Tags'].apply(ast.literal_eval) # Convert the string to a list.

tags_expanded = data['Tags'].apply(pd.Series) # Expand the 'Tags' column into multiple columns
tags_expanded.columns = [f'Tag_{i}' for i in range(tags_expanded.shape[1])] # Rename the columns for better readability

data = pd.concat([data, tags_expanded], axis=1) # Concatenate the expanded tags with the original dataframe

data.drop('Tags',axis=1,inplace=True)   # Drop the column 'Tags' due to the data being split into multiple columns

# The next part of the code turns the 'days_since_review' and 'Tag_3' columns into integers
data['days_since_review']=data['days_since_review'].str.extract('(\d+)').astype(int) # Extract the number from the string. This code if made using chatgpt
data['Tag_3'] = data['Tag_3'].str.extract('(\d+)').astype(float) # Extract the number from the string and convert to float. For some reason it did not work as int

# The next part of the code creates a new dataframe with hotel names and removes it from the dataframe data
hotel_names = data['Hotel_Name'] # Create a new dataframe with the hotel names
data.drop('Hotel_Name',axis=1,inplace=True) # Drop the column 'Hotel_Name' as it is not needed in the model

<unknown>:22: SyntaxWarning: invalid escape sequence '\d'
<unknown>:23: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<ipython-input-3-99ff9cf435f7>:22: SyntaxWarning: invalid escape sequence '\d'
  data['days_since_review']=data['days_since_review'].str.extract('(\d+)').astype(int) # Extract the number from the string. This code if made using chatgpt
<ipython-input-3-99ff9cf435f7>:23: SyntaxWarning: invalid escape sequence '\d'
  data['Tag_3'] = data['Tag_3'].str.extract('(\d+)').astype(float) # Extract the number from the string and convert to float. For some reason it did not work as int
<ipython-input-3-99ff9cf435f7>:22: SyntaxWarning: invalid escape sequence '\d'
  data['days_since_review']=data['days_since_review'].str.extract('(\d+)').astype(int) # Extract the number from the stri

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'Hotel_Reviews.csv'

### Model 1: Regressor

In [4]:
# Dropping columns not needed for this specific model
xgb_data = data.copy() # Create a copy of the dataframe data
xgb_data.drop('Reviewer_Nationality',axis=1,inplace=True) # Drop the column 'Revierer_Nationality' as it will give to many columns when one hot encoded
xgb_data.drop('Negative_Review',axis=1,inplace=True) # Drop the column 'Negative_Review' as it is not a numeric value
xgb_data.drop('Positive_Review',axis=1,inplace=True) # Drop the column 'Positive_Review' as it is not a numeric value

# One hot encode only the necessary columns
xgb_data = pd.get_dummies(xgb_data, columns=['Tag_0', 'Tag_1', 'Tag_2', 'Tag_4'], dtype=int)

# Turn the 'Reviewer_Score' column into a scale from 0-100 instead of 0-10
xgb_data['Reviewer_Score'] = xgb_data['Reviewer_Score'] * 10
xgb_data['Reviewer_Score']=xgb_data['Reviewer_Score'].astype(int) # Convert the 'Reviewer_Score' column to int

# The next part of the code splits the data into training and testing data. 
# The target can of course be changed to a bin type if needed.
x = xgb_data.drop('Reviewer_Score',axis=1) # Define the features
y = xgb_data['Reviewer_Score'] # Define the target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=69) # Split the data into training and testing data test_size is big due to memory issues when training the model

xgb_model = xgb.XGBRegressor(tree_method='hist',max_depth=3,n_estimators=50,gamma=0.2,subsample=0.8,colsample_bytree=0.8, learning_rate=0.05) # Create the xgboost model. The parameters are designed to limit the memory usage
xgb_model.fit(x_train, y_train) # Fit the model to the training data

guess=xgb_model.predict(x_test) # Make a guess on the testing data

<class 'NameError'>: name 'data' is not defined

### Model 2: K-Nearest Neighbors

In [5]:
knn_data = data.copy()
knn_data.dropna(inplace=True) # Drop the NaN in the colum
knn_data.drop(['Reviewer_Nationality', 'Negative_Review', 'Positive_Review'], axis=1, inplace=True)

# Grabbing a sample of the data for faster processing
percentage = 0.50
sample_size = int(knn_data.shape[0] * percentage)
knn_data_sample = knn_data.sample(n=sample_size, random_state=69)

features = knn_data_sample.columns.difference(['Average_Score'])
scaler = MinMaxScaler()
knn_data_sample[features] = scaler.fit_transform(knn_data_sample[features])

X = knn_data_sample.drop('Average_Score', axis=1) 
y = knn_data_sample['Average_Score'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'KNN Modellens Nøyaktighet: {mse:.4f}')

<class 'NameError'>: name 'data' is not defined

### Model 3: LSTM

In [6]:
# Denne cellen kan ignoreres. Brukes bare til å finne verdier for MAX_LEN og MAX_VOCAB
df = data.copy()

review_lengths = df["Review_Total_Negative_Word_Counts"] + df["Review_Total_Positive_Word_Counts"]

reviews = df["Negative_Review"] + df["Positive_Review"]

# Tokenize the reviews and create a list of all words
all_words = [word for review in reviews for word in review.lower().split()]

# Count the frequency of each word
word_counts = Counter(all_words)

# Calculate cumulative sum of word frequencies
sorted_word_counts = sorted(word_counts.values(), reverse=True)
cumulative_sum = np.cumsum(sorted_word_counts)
total_words = sum(sorted_word_counts)

# Find unique words by converting the list of tokens to a set
unique_words = set(all_words)

# Plot the distribution of review lengths (number of tokens)
plt.hist(review_lengths, bins=10, edgecolor='black')
plt.title('Distribution of Review Lengths (Tokens)')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.show()

# Printing relevant information
print(f"Max review length: {max(review_lengths)}")
print(f"Min review length: {min(review_lengths)}")
print(f"Average review length: {np.mean(review_lengths)}")
print(f"95th percentile review length: {np.percentile(review_lengths, 95)}")
print("\n")
print(f"Total number of unique words: {len(unique_words)}")

# Finding ideal MAX_VOCAB 
# Prioriterer i dette tilfellet coverage over andel av unike ord, fordi det er LSTM modell og det er liten forskjell i coverage mellom 10000 og 2500 ord
desired_coverage = 97.5

# Iterate over possible max_vocab values (or up to the length of cumulative_sum)
for max_vocab in range(1, len(cumulative_sum) + 1):
    # Get the cumulative sum for the top 'max_vocab' words
    coverage = cumulative_sum[max_vocab - 1]  # Take the cumulative sum at 'max_vocab' position
    
    # Calculate the coverage percentage
    coverage_percentage = (coverage / total_words) * 100
    
    # Check if the coverage has reached or exceeded the desired value
    if coverage_percentage >= desired_coverage:
        print(f"max_vocab for {desired_coverage}% coverage: {max_vocab} words")
        break  # Exit the loop as we've found the first match

<class 'NameError'>: name 'data' is not defined

In [7]:
# Data preprocessing

data_lstm = data.copy()

# Binary variable to determine sentiment (1 if Reviewer_Score >= 5, else 0)
data_lstm['positive'] = (data_lstm['Reviewer_Score'] >= 5).astype(int)

# Text preprocessing
MAX_VOCAB = 5018 # 97.5% coverage
MAX_LEN = 106 # 95th percentile of tokens in review

data_lstm['Positive_Review'] = data_lstm['Positive_Review'].astype(str)
data_lstm['Negative_Review'] = data_lstm['Negative_Review'].astype(str)


tok = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tok.fit_on_texts(data_lstm['Positive_Review'] + data_lstm['Negative_Review'])

# Convert text to sequences
X_pos = tok.texts_to_sequences(data_lstm['Positive_Review'])
X_neg = tok.texts_to_sequences(data_lstm['Negative_Review'])

# Pad sequences
X_pos = pad_sequences(X_pos, maxlen=MAX_LEN, padding='post', truncating='post')
X_neg = pad_sequences(X_neg, maxlen=MAX_LEN, padding='post', truncating='post')

# Numerical variables
num_variables = ['Average_Score', 'Review_Total_Negative_Word_Counts',
                'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given',
                'days_since_review']
scaler = StandardScaler()
X_num = scaler.fit_transform(data_lstm[num_variables])


# Label encoding used as opposed to one hot encoding, because it would be too memory intensive
# Label encode reviewer nationality
label_encoded_reviwer_nationality = LabelEncoder()
data_lstm['Reviewer_Nationality'] = label_encoded_reviwer_nationality.fit_transform(data_lstm['Reviewer_Nationality'])

# Convert 'Reviewer_Nationality' into a NumPy array and reshape for stacking
X_nationality = data_lstm['Reviewer_Nationality'].values.reshape(-1, 1)

tags = ['Tag_0', 'Tag_1', 'Tag_2', 'Tag_3', 'Tag_4', 'Tag_5']
# Label encode each tag column individually
for tag in tags:
    label_encoded_tags = LabelEncoder()
    data_lstm[tag] = label_encoded_tags.fit_transform(data_lstm[tag])  # Convert each tag into integer labels

X_tags = data_lstm[tags].values  # Convert to NumPy array

# Combine features
X = np.hstack((X_pos, X_neg, X_num, X_nationality, X_tags))
y = data_lstm['positive'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # stratify = y because of high level of imbalance in positive and negative reviews


# print(data_lstm.dtypes)  # Check overall data types
# print(data_lstm[['Reviewer_Nationality']].dtypes)  # Check the label-encoded column
# print(data_lstm[tags].dtypes)  # Check the tag columns (still one-hot encoded)

# print(Counter(data_lstm['positive'])) # Check ratio of positive to negative reviews

<class 'NameError'>: name 'data' is not defined

In [8]:
# Build the LSTM model
model = Sequential([
    Embedding(MAX_VOCAB, 128, input_length=X_train.shape[1]),
    LSTM(64, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.2), # Low dropout as overfitting is near 0 anyway
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])

# Train model
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_data=(X_test, y_test))

# The model converges quickly, and performance is good regardless of batch size, so only 2-3 epochs needed with 

<class 'NameError'>: name 'Sequential' is not defined

In [9]:
# Used for plotting the amount of positive and negative reviews predicted by the model

# Get raw predictions (probabilities)
y_pred_probs = model.predict(X_test)

# Convert probabilities to binary labels (positive or negative review)
y_pred = (y_pred_probs >= 0.5).astype(int)

positive_count = np.sum(y_pred == 1)
negative_count = np.sum(y_pred == 0)

categories = ['Negative', 'Positive']
counts = [negative_count, positive_count]

plt.bar(categories, counts, color=['red', 'green'])
plt.xlabel("Review Sentiment")
plt.ylabel("Number of Predictions")
plt.title("Distribution of Predicted Sentiments")

plt.show()

<class 'NameError'>: name 'model' is not defined

In [10]:
# This cell is for evaluating the model

test_loss, test_accuracy, test_auc = model.evaluate(X_test, y_test)
train_loss, train_accuracy, train_auc = model.evaluate(X_train, y_train)
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'Train accuracy: {train_accuracy:.2f}')
print(f'Overfitting: {train_accuracy-test_accuracy:.2f}')

# Convert predicted probabilities to binary classification (positive or negative review)
y_pred_prob = model.predict(X_test)  
y_pred = (y_pred_prob >= 0.5).astype(int)  # Convert to binary classification

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob) 

# Results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}') # How many predicted positives are actually positive
print(f'Recall: {recall:.2f}') # How many actual positive cases were correctly predicted
print(f'F1-Score: {f1:.2f}') # Balance betweet precision and recall
print(f'ROC-AUC: {roc_auc:.2f}') # Area under ROC curve (recall against false positive rate)

# Scores from 0-1, where higher scores = better performance

<class 'NameError'>: name 'model' is not defined

### Model 4: TF-IDF

In [11]:
tfidf_data = pd.read_csv('data/Hotel_Reviews.csv')

#reviews = data["Positive_Review"]
reviews = tfidf_data["Negative_Review"]

tfidf_vectorizer = TfidfVectorizer(stop_words="english")

X = tfidf_vectorizer.fit_transform(reviews)
y = tfidf_data["Reviewer_Score"].map(lambda x: 0 if x < 5.0 else 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'data/Hotel_Reviews.csv'